<a href="https://colab.research.google.com/github/k2-fsa/colab/blob/master/sherpa-onnx/itn_zh_for_HA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

本 colab notebook 演示如何生成 rule fst, 把匹配到的短语，换成自己想要的结果。
你可以用生成的 fst，结合 [kaldifst](https://github.com/k2-fsa/kaldifst) 进行部署。 提供 C++ 和 Python 等 API.

如何在语音识别中使用生成的 fst, 请参考 [sherpa-onnx](https://github.com/k2-fsa/sherpa-onnx)



# Install pynini

In [1]:
%%shell

pip install --only-binary :all: pynini

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 MB 4.9 MB/s eta 0:00:00


# Generate rule fst

In [9]:
import pynini
from pynini.lib import utf8
from pynini.lib.pynutil import add_weight, delete, insert


sigma = utf8.VALID_UTF8_CHAR.star


phrases = [
    # 识别的结果,     想要的结果
    ('打开客厅通灯', '打开客厅筒灯'),
    ('打开客厅同灯', '打开客厅筒灯'),
]

rules = pynini.string_map(phrases).optimize()


rules = pynini.cdrewrite(rules, "", "", sigma)

# 下面的 for 语句是一个测试，可以删掉
for d in ['打开客厅通灯',
          '通灯', # 需要完整匹配，所以，这个词还是它本身
          '打开客厅同灯',
          ]:
  r = pynini.compose(d, rules)
  s = pynini.shortestpath(r, nshortest=1).paths()
  print(d, list(s.ostrings())[:3])

rules.write('itn_zh_HA.fst')

! ls -lh itn_zh_HA.fst

打开客厅通灯 ['打开客厅筒灯']
通灯 ['通灯']
打开客厅同灯 ['打开客厅筒灯']
-rw-r--r-- 1 root root 38K Jun 25 01:51 itn_zh_HA.fst


In [10]:
# 下载到本地
from google.colab import files
files.download('itn_zh_HA.fst')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Usage with kaldifst

In [12]:
%%shell

pip install kaldifst

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 31.4 MB/s eta 0:00:00


In [1]:
import kaldifst

InverseTextNormalizer = kaldifst.TextNormalizer

rule = "./itn_zh_HA.fst"
normalizer = InverseTextNormalizer(rule)
text = "我要打开客厅通灯"
out = normalizer(text)
print(out)

我要打开客厅筒灯
